Importemos cosas:

In [1]:
import numpy as np
from qutip import *
from itertools import combinations
from scipy.integrate import quad

### 1.

#### 1a.
Veamos si la base es ortonormal:

In [6]:
#   Definr las funciones
fn = [
    lambda x: 1,
    lambda x: np.sin(np.pi*x),
    lambda x: np.cos(np.pi*x),
    lambda x: np.sin(2*np.pi*x),
    lambda x: np.cos(2*np.pi*x)
]

(a,b) = -1,1

#   Calcular producto interno
for i, j in combinations(fn, 2):
    inner_p = quad(lambda x: i(x)*j(x),-1,1)[0]
    if np.isclose(inner_p,0):
        flag = 1
        continue
    else:
        flag = 0
        pass
    break

if flag==0: print('El conjunto no es ortogonal\n')
else:   print('El conjunto es ortogonal\n')

#   Calcular la magnitud
for i,f in enumerate(fn):
    print(f'La integral de |f{i+1}(x)|^2 en [{a}, {b}] = {quad(lambda x: f(x)**2,-1,1)[0]:.1f}')


El conjunto es ortogonal

La integral de |f1(x)|^2 en [-1, 1] = 2.0
La integral de |f2(x)|^2 en [-1, 1] = 1.0
La integral de |f3(x)|^2 en [-1, 1] = 1.0
La integral de |f4(x)|^2 en [-1, 1] = 1.0
La integral de |f5(x)|^2 en [-1, 1] = 1.0


Para normalizar $f1(x)$ es suficiente multiplicar por su factor de normalización $\frac{1}{\sqrt{2}}$.

In [10]:
#   Definir nueva f1(x)
fn[0] = lambda x: 1/np.sqrt(2)

#   Calcular la magnitud, otra vez
for i,f in enumerate(fn):
    print(f'La integral de |f{i+1}(x)|^2 en [{a}, {b}] = {quad(lambda x: f(x)**2,-1,1)[0]:.1f}')

La integral de |f1(x)|^2 en [-1, 1] = 1.0
La integral de |f2(x)|^2 en [-1, 1] = 1.0
La integral de |f3(x)|^2 en [-1, 1] = 1.0
La integral de |f4(x)|^2 en [-1, 1] = 1.0
La integral de |f5(x)|^2 en [-1, 1] = 1.0


#### 1b,1c.
Construyamos la base y kets:

In [15]:
#   Construir base
o1 = Qobj([[1],[0],[0],[0],[0]])
s1 = Qobj([[0],[1],[0],[0],[0]])
c1 = Qobj([[0],[0],[1],[0],[0]])
s2 = Qobj([[0],[0],[0],[1],[0]])
c2 = Qobj([[0],[0],[0],[0],[1]])

# Construir kets
fk = [
    (1/np.sqrt(5))*(o1+s1+c1+s2+c2),
    (1/np.sqrt(3))*(o1 + s1 + c1),
    (1/np.sqrt(2))*(s2+c2)
]

# Construir bras
fb = [f.dag() for f in fk]

Evaluemos $\langle fi|fi\rangle \;  (i=1,2,3)$; y $\langle f1|f2\rangle$, $\langle f1|f3\rangle$ y $\langle f2|f3\rangle$:

In [28]:
#   Normalización
for i,f in enumerate(fb):    print(f'<f{i+1}|f{i+1}> = {np.sqrt(f.overlap(fk[i]))}')

#   Ortogonalidad
print(f'\n<f{1}|f{2}> = {fb[0].overlap(fk[1])}')
print(f'<f{1}|f{3}> = {fb[0].overlap(fk[2])}')
print(f'<f{2}|f{3}> = {fb[1].overlap(fk[2])}')

<f1|f1> = (0.9999999999999999+0j)
<f2|f2> = (1+0j)
<f3|f3> = (0.9999999999999999+0j)

<f1|f2> = (0.7745966692414834+0j)
<f1|f3> = (0.6324555320336758+0j)
<f2|f3> = 0j


Las funciones se encuentran normalizadas, pero únicamente $f2$ y $f3$ son ortogonales.

### 2.
Construyamos los proyectores:

In [30]:
#   Proyectores a partir de kets
proj = [o1.proj(),s1.proj(),c2.proj()]
print(f'Tipo: {proj[0].type}')

Tipo: oper


Estos objetos son operadores.

### 3.
Evaluemos $|1\rangle \langle 1| f2 \rangle$, $|s1\rangle \langle s1| f2 \rangle$ y $|c2\rangle \langle c2| f2 \rangle$:

In [39]:
labels = ('1','s1','c2')
for p,l in zip(proj,labels):  print(f'|{l}><{l}|f2>:\n{(p*fk[1])}\n')

|1><1|f2>:
Quantum object: dims = [[5], [1]], shape = (5, 1), type = ket
Qobj data =
[[0.57735027]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]]

|s1><s1|f2>:
Quantum object: dims = [[5], [1]], shape = (5, 1), type = ket
Qobj data =
[[0.        ]
 [0.57735027]
 [0.        ]
 [0.        ]
 [0.        ]]

|c2><c2|f2>:
Quantum object: dims = [[5], [1]], shape = (5, 1), type = ket
Qobj data =
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]]



Todos los resultados son nuevos kets. El resultado de $|c2\rangle \langle c2| f2 \rangle$ es $0$ pues $| f2 \rangle$ no tiene componentes proyectables sobre $|c2\rangle \langle c2|$.

### 4.
Tenemos que

$|s1\rangle \langle s1 |c2\rangle \langle c2| f2 \rangle$

Entonces

$\left ( |s1\rangle \langle s1 |c2\rangle \langle c2|\right )| f2 \rangle$

Y evaluando:

In [41]:
print(f'|s1><s1|c2><c2|:\n{(proj[1]*proj[2])}\n')

|s1><s1|c2><c2|:
Quantum object: dims = [[5], [5]], shape = (5, 5), type = oper, isherm = True
Qobj data =
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]



Dado que este operador nuevo es $0$, aplicarlo sobre $|f2\rangle$ resultaría en el estado nulo.